# Merge BA Loads

This notebook merges together the time series of hourly electricity demand from transportation and non-transportation sources by Balancing Authority (BA).

In [ ]:
# Start by importing the packages we need:
import os
import glob

import pandas as pd


## Set the Directory Structure

In [ ]:
# Identify the data input and output directories:
trans_data_input_dir =  '/Users/burl878/Documents/IMMM/GODEEP/Data/Transportation/Combined/'
tell_data_input_dir =  '/Users/burl878/Documents/IMMM/GODEEP/Data/TELL/Production_Runs/tell_data/outputs/tell_output/'
data_output_dir =  '/Users/burl878/Documents/IMMM/GODEEP/Data/Merged_BA_Loads/'


## Set the Scenario and Year You Want to Process

In [ ]:
scenario_to_process = 'BAU_Climate'
year_to_process = '2025'


## Merge the Transportation Output Files Together

In [ ]:
def merge_transportation_data(scenario: str, year: str, trans_data_input_dir: str, data_output_dir: str):
    # Check to see if the processed output file exist and if not then create it:
    if os.path.isfile((os.path.join(data_output_dir, ('Transportation_Load_Time_Series_' + scenario + '_' + year + '.csv')))) == False:
       
       # Create a list of all of the transportation output files in the "trans_data_input_dir" and aggregate the files in that list:
       trans_filelist = sorted(glob.glob(os.path.join(trans_data_input_dir, scenario, ('*' + year + '.csv'))))
       
       # Loop over the list of files:
       for file in range(len(trans_filelist)):
           # Read in the .csv file:
           trn_data = pd.read_csv(trans_filelist[file])

           # Rename a few variables for consistency with TELL:
           trn_data.rename(columns={'balancing_authority': 'BA_Code',
                                    'time': 'Time_UTC',
                                    'transportation_load_MWh': 'Transportation_Load_MWh'}, inplace=True)
        
           # Only keep the columns that are needed:
           trn_data = trn_data[['BA_Code', 'Time_UTC', 'Transportation_Load_MWh']].copy()
           
           # Strip the "+00:00" from the time string:
           trn_data['Time_UTC'] = trn_data['Time_UTC'].astype(str)
           trn_data['Time_UTC'] = trn_data['Time_UTC'].str.split('+').str[0]

           # Aggregate the output into a new dataframe:
           if file == 0:
              trn_output_df = trn_data
           else:
              trn_output_df = pd.concat([trn_output_df, trn_data])
         
           # Clean up and move to the next file:
           del trn_data
           
       # Write out the dataframe to a .csv file:
       trn_output_df.to_csv((os.path.join(data_output_dir, ('Transportation_Load_Time_Series_' + scenario + '_' + year + '.csv'))), sep=',', index=False)
   
    else:
       # Read in the already processed output file:
       trn_output_df = pd.read_csv((os.path.join(data_output_dir, ('Transportation_Load_Time_Series_' + scenario + '_' + year + '.csv'))))            
                  
    return trn_output_df


In [ ]:
# Merge the transportation output files into a single dataframe:
trn_df = merge_transportation_data(scenario = scenario_to_process, 
                                   year = year_to_process, 
                                   trans_data_input_dir = trans_data_input_dir, 
                                   data_output_dir = data_output_dir)

# Preview the transportation dataframe:
trn_df


## Read in the TELL Output File and Subset to Just the BAs in the WECC


In [ ]:
# Read in the BA-to-Interconnection mapping file:
ba_mapping_df = pd.read_csv(data_output_dir + 'BA_to_Interconnection_Mapping.csv')


In [ ]:
# Read in the TELL BA output file for the year and scenario being processed:
tell_df = pd.read_csv(tell_data_input_dir + scenario_to_process + '/' + year_to_process + '/TELL_Balancing_Authority_Hourly_Load_Data_' + year_to_process + '_Scaled_' + year_to_process + '.csv')

# Rename a few variables for consistency:
tell_df.rename(columns={'Scaled_TELL_BA_Load_MWh': 'Non-Transportation_Load_MWh'}, inplace=True)

# Merge the ba_mapping_df and tell_df dataframes based on common "BA_Code":
tell_df = tell_df.merge(ba_mapping_df, on=['BA_Code'])

# Subset to just the WECC BAs:
tell_df = tell_df[tell_df['Interconnection'] == 'WECC'].copy()

# Make the time variable a string:
tell_df['Time_UTC'] = tell_df['Time_UTC'].astype(str)

# Only keep the columns that are needed:
tell_df = tell_df[['BA_Code', 'Time_UTC', 'Non-Transportation_Load_MWh']].copy()

# Preview the TELL dataframe:
tell_df


## Merge the TELL and Transportation Dataframes Together


In [ ]:
# Merge the trn_df and tell_df dataframes based on common "BA_Code" and "Time_UTC":
merged_df = tell_df.merge(trn_df, on=['BA_Code', 'Time_UTC'])

# Compute the sum of the transportation and non-transportation loads:
merged_df['Total_Load_MWh'] = merged_df['Non-Transportation_Load_MWh'] + merged_df['Transportation_Load_MWh']

# Round off the values to make the output file more readable:
merged_df['Non-Transportation_Load_MWh'] = merged_df['Non-Transportation_Load_MWh'].round(2)
merged_df['Transportation_Load_MWh'] = merged_df['Transportation_Load_MWh'].round(2)
merged_df['Total_Load_MWh'] = merged_df['Total_Load_MWh'].round(2)

# Fill in missing values with -9999:
merged_df['Non-Transportation_Load_MWh'] = merged_df['Non-Transportation_Load_MWh'].fillna(-9999)
merged_df['Transportation_Load_MWh'] = merged_df['Transportation_Load_MWh'].fillna(-9999)
merged_df['Total_Load_MWh'] = merged_df['Total_Load_MWh'].fillna(-9999)

# Write out the dataframe to a .csv file:
merged_df.to_csv((os.path.join(data_output_dir, ('Total_Load_Time_Series_' + scenario_to_process + '_' + year_to_process + '.csv'))), sep=',', index=False)

# Preview the merged dataframe:
merged_df
